In [2]:
from tqdm import tqdm
from collections import defaultdict
from copy import deepcopy
import os
from pprint import pprint

import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import numpy as np
import cv2
from mmcv import imread
from mmengine import dump, load

from utils import box_xywh2xyxy, box_xyxy2xywh

In [3]:
train_img = load('../../annotations/image_info_train.json')
test_img  = load('../../annotations/image_info_test.json')

train_img_root = '../../images/train/'
test_img_root = '../../images/test/'

ins_categories = load('../../configs/ins_categories.json')
coco_ins_label2cat = {i: cat['id'] for i, cat in enumerate(ins_categories)}
coco_ins_cat2label = {cat['id']: i for i, cat in enumerate(ins_categories)}

# 方法

## Instance

### watch

In [4]:
font = cv2.FONT_HERSHEY_SIMPLEX
palette = np.asarray([
    (220, 20, 60), (119, 11, 32), (0, 0, 142), (0, 0, 230), (106, 0, 228),
    (0, 60, 100), (0, 80, 100), (0, 0, 70), (0, 0, 192), (250, 170, 30),
    (100, 170, 30), (220, 220, 0), (175, 116, 175), (250, 0, 30),
    (165, 42, 42), (255, 77, 255), (0, 226, 252), (182, 182, 255),
    (0, 82, 0), (120, 166, 157), (110, 76, 0), (174, 57, 255),
    (199, 100, 0), (72, 0, 118), (255, 179, 240), (0, 125, 92),
    (209, 0, 151), (188, 208, 182), (0, 220, 176), (255, 99, 164),
    (92, 0, 73), (133, 129, 255), (78, 180, 255), (0, 228, 0),
    (174, 255, 243), (45, 89, 255), (134, 134, 103), (145, 148, 174),
    (255, 208, 186), (197, 226, 255), (171, 134, 1), (109, 63, 54),
    (207, 138, 255), (151, 0, 95), (9, 80, 61), (84, 105, 51),
    (74, 65, 105), (166, 196, 102), (208, 195, 210), (255, 109, 65),
    (0, 143, 149), (179, 0, 194), (209, 99, 106), (5, 121, 0),
    (227, 255, 205), (147, 186, 208), (153, 69, 1), (3, 95, 161),
    (163, 255, 0), (119, 0, 170), (0, 182, 199), (0, 165, 120),
    (183, 130, 88), (95, 32, 0), (130, 114, 135), (110, 129, 133),
    (166, 74, 118), (219, 142, 185), (79, 210, 114), (178, 90, 62),
    (65, 70, 15), (127, 167, 115), (59, 105, 106), (142, 108, 45),
    (196, 172, 0), (95, 54, 80), (128, 76, 255), (201, 57, 1),
    (246, 0, 122), (191, 162, 208)], dtype=float) / 255

In [5]:
def show_box(box, ax, info: dict = {}):
    x, y, w, h = box

    edgecolor = info.get('edgecolor', 'green')
    name = info.get('name', '')

    # ax.text(x0, y0, name, fontdict=font)
    ax.text(x, y, name)
    # 底色
    ax.add_patch(plt.Rectangle((x, y), w, h, edgecolor='white', facecolor=(0,0,0,0), lw=2))
    # 类别颜色
    ax.add_patch(plt.Rectangle((x, y), w, h, edgecolor=edgecolor, facecolor=(0,0,0,0), lw=1))

def watch_ins_proposal(proposal: dict, img_path: str, with_name: bool = True):
    image = imread(img_path, channel_order='rgb')
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.imshow(image)
    ax.set_title(os.path.basename(img_path))
    ax.axis('off')
    bboxes = proposal['instances']['bboxes']
    labels = proposal['instances']['labels']
    for box, label in zip(bboxes, labels):
        cat = coco_ins_label2cat[label]
        info = dict(edgecolor=palette[label])
        if with_name:
            info['name'] = ins_categories[cat]['name']
        show_box(box, ax, info=info)

### convert

In [6]:
def cvt_mmdet_proposal(ins_pred: list, coco_img: dict, score_thr: float = 0.1):
    """从mmdet的预测转换"""
    id2filename = {item['id']: item['file_name'] for item in coco_img['images']}

    num_valid = 0
    ins_proposals = defaultdict(lambda:{'bboxes': [], 'labels': [], 'scores': []})
    for ins in tqdm(ins_pred):
        if ins['score'] < score_thr:
            continue
        num_valid += 1
        img_name = id2filename[ins['image_id']]
        ins_proposals[img_name]['bboxes'].append(box_xywh2xyxy(ins['bbox']))
        ins_proposals[img_name]['labels'].append(ins['category_id'])
        ins_proposals[img_name]['scores'].append(ins['score'      ])
    print('num valid:', num_valid)

    total_proposals = {}
    for filename, props in ins_proposals.items():
        labels = props['labels']
        if coco_ins_cat2label is not None:
            labels = [coco_ins_cat2label[i] for i in labels]
        ins_props = {
            'bboxes': np.float32(props['bboxes']).round(0),
            'scores': np.float32(props['scores']).round(3),
            'labels': np.int64(labels),
            'ids'   : np.arange(len(labels))  # 单张图内id
        }

        total_proposals[filename] = {'instances': ins_props}
    return total_proposals

In [7]:
def cvt_scg_proposal(root, ins_label_2_id_map: dict = None, 
                     score_thr: float = 0.1, human_id: int = 0):
    """从SCG系列（SCG, ViPLO, UPT）的proposal转换"""
    total_proposals = {}
    for filename in tqdm(os.listdir(root)):
        props = load(os.path.join(root, filename))
        img_name = filename.replace('.json', '.jpg')

        labels = props['labels']
        if ins_label_2_id_map is not None:
            labels = [ins_label_2_id_map[i] for i in labels]

        bboxes = np.float32(props['boxes' ]).round(0)  # xyxy
        scores = np.float32(props['scores']).round(3)
        labels = np.int64(labels)

        keep = scores > score_thr
        keep_human = keep[labels == human_id]

        ins_props = {
            'bboxes': bboxes[keep],
            'scores': scores[keep],
            'labels': labels[keep],
            'ids'   : np.arange(keep.sum())  # 单张图内id
        }

        if 'human_joints' in props:
            points = np.float32(props['human_joints']
                                ).round(0).reshape(-1, 17, 2)
            scores = np.float32(props['human_joints_score']
                                ).round(3).reshape(-1, 17)
            pose_props = {
                'keypoints' : points[keep_human],
                'scores'    : scores[keep_human],
                'tgt_ins_id': np.where(ins_props['labels']==human_id)[0]
            }

            total_proposals[img_name] = {
                'instances'       : ins_props, 
                'person_keypoints': pose_props
            }
        else:
            total_proposals[img_name] = {'instances': ins_props}
    return total_proposals

In [8]:
def cvt_ins_ann2proposal(coco_ins: COCO):
    proposals = dict()
    for img_id in tqdm(coco_ins.getImgIds()):
        img_info = coco_ins.loadImgs(img_id)[0]
        ins_anns = coco_ins.loadAnns(sorted(coco_ins.getAnnIds(img_id)))

        bboxes, labels = [], []
        for ann in ins_anns:
            bboxes.append(box_xywh2xyxy(ann['bbox']))
            labels.append(coco_ins_cat2label[ann['category_id']])

        proposals[img_info['file_name']] = {
            'instances': {
                'bboxes': np.float32(bboxes),
                'scores': np.ones_like(labels, dtype=np.float32),
                'labels': np.int64(labels),
                'ids'   : np.arange(len(labels))  # 单张图内id
            }
        }
    return proposals

## Person keypoints

### watch

In [9]:
from mmengine.structures import InstanceData
from mmpose.structures import PoseDataSample
from mmpose.visualization import PoseLocalVisualizer

/home/wuhao/.conda/envs/torch2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wuhao/.conda/envs/torch2/lib/python3.9/site-packages/colossalai/kernel/cuda_native/mha/flash_attn_2.py:21: UserWarning: FlashAttention only supports Ampere GPUs or newer.
  warnings.warn("FlashAttention only supports Ampere GPUs or newer.")
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0)
    Python  3.9.18 (you have 3.9.18)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [10]:
pose_categories = load('../../configs/pose_categories.json')
pprint(pose_categories, compact=True)

dataset_meta = {'skeleton_links': pose_categories[0]['skeleton']}
pose_local_visualizer = PoseLocalVisualizer(
    link_color=tuple((255, 0, 0)) * len(dataset_meta['skeleton_links']),
    line_width=4)
pose_local_visualizer.set_dataset_meta(dataset_meta)

[{'id': 1,
  'keypoints': ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear',
                'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
                'left_wrist', 'right_wrist', 'left_hip', 'right_hip',
                'left_knee', 'right_knee', 'left_ankle', 'right_ankle'],
  'name': 'person',
  'skeleton': [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12],
               [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3],
               [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]],
  'supercategory': 'person'}]


In [11]:
def watch_pose_proposal(proposal: dict, img_path: str):
    visualizer = deepcopy(pose_local_visualizer)
    image = imread(img_path, channel_order='rgb')
    data = PoseDataSample()
    keypoints = np.array(proposal['person_keypoints']['keypoints']).reshape(-1, 17, 3)
    data.gt_instances = InstanceData(keypoints=keypoints)
    vis_result = visualizer.add_datasample('image', image, data, draw_pred=False)
    plt.imshow(vis_result)

### convert

In [12]:
def extract_person_from_proposal(ins_props: dict, coco_img: dict):
    """挑出“人”实例，用于姿态估计"""
    filename2id = {item['file_name']: item['id'] for item in coco_img['images']}

    human_ins = []
    imgs_wo_human = []

    for filename, prop in tqdm(ins_props.items()):
        prop = prop['instances']
        image_id = filename2id[filename]
        wo_human = True
        for label, bbox, score, id in zip(prop['labels'], prop['bboxes'], 
                                          prop['scores'], prop['ids'   ]):
            if label == 0:
                wo_human = False
                human_ins.append({
                    'image_id'   : image_id,
                    'bbox'       : box_xyxy2xywh(bbox),
                    'score'      : score,
                    'category_id': 1,
                    'id'         : id
                })
        if wo_human:
            imgs_wo_human.append(image_id)

    print('num person:', len(human_ins))
    print('num image wo person:', len(imgs_wo_human))
    return human_ins, imgs_wo_human

## 合并各种类型的proposal

In [13]:
def merge_pose_to_prop(proposals: dict, pose_pred: list, coco_img: dict) -> dict:
    id2filename = {item['id']: item['file_name'] for item in coco_img['images']}

    pose_proposals = defaultdict(list)
    for pose in tqdm(pose_pred):
        img_name = id2filename[pose['image_id']]
        pose_proposals[img_name].append(pose['keypoints'])
    pose_proposals = dict(pose_proposals)

    print('num image:', len(set(proposals.keys())))
    print('num image wo pose prop:', 
          len(set(proposals.keys()) ^ set(pose_proposals.keys())))

    total_proposals = deepcopy(proposals)
    for img_name, props in total_proposals.items():
        if img_name not in pose_proposals:
            # 没有person预测，也就没有pose预测
            poses = np.zeros((0, 17, 3))
        else:
            poses = pose_proposals[img_name]
            poses = np.array(poses, dtype=np.float32).reshape(-1, 17, 3)
        props['person_keypoints'] = {
            'keypoints': poses[..., :-1],
            'scores'   : poses[...,  -1],
            'tgt_ins_id': np.where(proposals[img_name]['instances']['labels']==0)[0]
        }
    return total_proposals

# 实际转换

### FasterRCNN(SCG)

In [21]:
proposal_root = '../../proposals/SCG/'
train_total_prop_path = os.path.join(proposal_root, 'proposal_scg_train.pkl')
test_total_prop_path  = os.path.join(proposal_root, 'proposal_scg_test.pkl')
train_ins_prop_root   = os.path.join(proposal_root, 'train2015')
test_ins_prop_root    = os.path.join(proposal_root, 'test2015')
train_pose_prop_path  = os.path.join(proposal_root, 'body2d_hico-train_scg.keypoints.json')
test_pose_prop_path   = os.path.join(proposal_root, 'body2d_hico-test_scg.keypoints.json')
train_person_path     = os.path.join(proposal_root, 'train_person.json')
test_person_path      = os.path.join(proposal_root, 'test_person.json')

#### instance

In [22]:
coco_ins_labels = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 
    'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 
    'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 
    'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 
    'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 
    'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
scg_ins_labels = [
    'airplane', 'apple', 'backpack', 'banana', 'baseball bat', 'baseball glove',
    'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl',
    'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair',
    'clock', 'couch', 'cow', 'cup', 'dining table', 'dog', 'donut', 'elephant',
    'fire hydrant', 'fork', 'frisbee', 'giraffe', 'hair drier', 'handbag', 'horse',
    'hot dog', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorcycle',
    'mouse', 'orange', 'oven', 'parking meter', 'person', 'pizza', 'potted plant',
    'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink',
    'skateboard', 'skis', 'snowboard', 'spoon', 'sports ball', 'stop sign',
    'suitcase', 'surfboard', 'teddy bear', 'tennis racket', 'tie', 'toaster',
    'toilet', 'toothbrush', 'traffic light', 'train', 'truck', 'tv', 'umbrella',
    'vase', 'wine glass', 'zebra']
scg_ins_map = {}
for i, key in enumerate(scg_ins_labels):
    j = coco_ins_labels.index(key)
    scg_ins_map[i] = j

In [23]:
# 转换proposal格式
train_proposals = cvt_scg_proposal(train_ins_prop_root, scg_ins_map, score_thr=0.1)
test_proposals  = cvt_scg_proposal( test_ins_prop_root, scg_ins_map, score_thr=0.1)

  0%|          | 0/37633 [00:00<?, ?it/s]

100%|██████████| 9546/9546 [00:01<00:00, 6004.31it/s]


In [ ]:
# train_proposals = load(train_total_prop_path)
# test_proposals  = load( test_total_prop_path)
# len(train_proposals), len(test_proposals)

In [24]:
train_proposals['HICO_train2015_00000001.jpg']

{'instances': {'bboxes': array([[208.,  26., 455., 321.],
         [ 46., 101., 579., 397.]], dtype=float32),
  'scores': array([1.   , 0.999], dtype=float32),
  'labels': array([0, 3]),
  'ids': array([0, 1])}}

#### person_keypoints

In [25]:
# 提取仅含“人”的实例
train_person, train_imgs_wo_human = extract_person_from_proposal(train_proposals, train_img)
test_person ,  test_imgs_wo_human = extract_person_from_proposal( test_proposals,  test_img)
dump(train_person, train_person_path)
dump( test_person,  test_person_path)

  0%|          | 0/37633 [00:00<?, ?it/s]

100%|██████████| 37633/37633 [00:01<00:00, 32375.15it/s]


num person: 220261
num image wo person: 14


100%|██████████| 9546/9546 [00:00<00:00, 23077.69it/s]


num person: 54433
num image wo person: 19


In [ ]:
# train_person = load(train_person_path)
# test_person  = load( test_person_path)
# len(train_person), len(test_person)

In [29]:
print(train_person[10])

{'image_id': 16930, 'bbox': [63.0, 115.0, 30.0, 62.0], 'score': 0.365, 'category_id': 1, 'id': 11}


In [ ]:
# 获取姿态估计结果
train_pose = load(train_pose_prop_path)
test_pose  = load( test_pose_prop_path)
assert len(train_pose) == len(train_person), f'{len(train_pose)} != {len(train_person)}'
assert len( test_pose) == len( test_person), f'{len( test_pose)} != {len( test_person)}'

In [ ]:
# 转换成keypoints_proposal
train_proposals = merge_pose_to_prop(train_proposals, train_pose, train_img)
test_proposals  = merge_pose_to_prop( test_proposals,  test_pose,  test_img)

In [147]:
dump(train_proposals, train_total_prop_path)
dump( test_proposals,  test_total_prop_path)

### UPT(ViPLO)

In [31]:
proposal_root = '../../proposals/ViPLO/'
train_total_prop_path = os.path.join(proposal_root, 'proposal_viplo_train.pkl')
test_total_prop_path  = os.path.join(proposal_root, 'proposal_viplo_test.pkl')
train_ins_prop_root   = os.path.join(proposal_root, 'train2015_vitpose')
test_ins_prop_root    = os.path.join(proposal_root, 'test2015_upt_vitpose')
# train_pose_prop_path  = os.path.join(proposal_root, 'body2d_hico-train_upt.keypoints.json')  # 含在train_ins_prop_root中
# test_pose_prop_path   = os.path.join(proposal_root, 'body2d_hico-test_upt.keypoints.json')
train_person_path     = os.path.join(proposal_root, 'train_person.json')
test_person_path      = os.path.join(proposal_root, 'test_person.json')

#### instance & person_keypoints

In [32]:
# 转换proposal格式
train_proposals = cvt_scg_proposal(train_ins_prop_root, scg_ins_map, score_thr=0.1)
test_proposals  = cvt_scg_proposal( test_ins_prop_root, scg_ins_map, score_thr=0.1)

  0%|          | 0/37633 [00:00<?, ?it/s]

100%|██████████| 9546/9546 [00:06<00:00, 1578.63it/s]


In [ ]:
# train_proposals = load(train_total_prop_path)
# test_proposals  = load( test_total_prop_path)
# len(train_proposals), len(test_proposals)

In [36]:
train_proposals['HICO_train2015_00000001.jpg']

{'instances': {'bboxes': array([[ 54.,  82., 581., 404.],
         [212.,  37., 438., 295.]], dtype=float32),
  'scores': array([0.999, 0.986], dtype=float32),
  'labels': array([3, 0]),
  'ids': array([0, 1])},
 'person_keypoints': {'keypoints': array([[[401., 101.],
          [407.,  93.],
          [394.,  92.],
          [412.,  89.],
          [371.,  77.],
          [384., 108.],
          [323.,  96.],
          [424., 138.],
          [311., 162.],
          [415., 165.],
          [370., 174.],
          [303., 160.],
          [249., 142.],
          [349., 194.],
          [283., 164.],
          [327., 280.],
          [238., 255.]]], dtype=float32),
  'scores': array([[0.842, 0.897, 0.953, 0.391, 0.822, 0.423, 0.85 , 0.335, 0.917,
          0.418, 0.792, 0.466, 0.769, 0.4  , 0.737, 0.369, 0.828]],
        dtype=float32),
  'tgt_ins_id': array([1])}}

In [37]:
# 提取仅含“人”的实例
train_person, train_imgs_wo_human = extract_person_from_proposal(train_proposals, train_img)
test_person ,  test_imgs_wo_human = extract_person_from_proposal( test_proposals,  test_img)
dump(train_person, train_person_path)
dump( test_person,  test_person_path)

100%|██████████| 37633/37633 [00:02<00:00, 16419.31it/s]


num person: 376085
num image wo person: 125


100%|██████████| 9546/9546 [00:00<00:00, 45665.08it/s]


num person: 82428
num image wo person: 3


In [ ]:
# train_person = load(train_person_path)
# test_person  = load( test_person_path)
# len(train_person), len(test_person)

In [39]:
print(train_person[100])

{'image_id': 21540, 'bbox': [287.0, 116.0, 25.0, 35.0], 'score': 0.107, 'category_id': 1, 'id': 81}


In [140]:
dump(train_proposals, train_total_prop_path)
dump( test_proposals,  test_total_prop_path)

## Ground Truth

In [40]:
proposal_root = '../../proposals/GT/'
train_total_prop_path = os.path.join(proposal_root, 'proposal_gt_train.pkl')
test_total_prop_path  = os.path.join(proposal_root, 'proposal_gt_test.pkl')
train_pose_prop_path  = os.path.join(proposal_root, 'body2d_hico-train_gt.keypoints.json')
test_pose_prop_path   = os.path.join(proposal_root, 'body2d_hico-test_gt.keypoints.json')
train_person_path     = os.path.join(proposal_root, 'train_person.json')
test_person_path      = os.path.join(proposal_root, 'test_person.json')

### instance

In [130]:
train_ins = COCO('../../annotations/instances_train.json')
test_ins  = COCO('../../annotations/instances_test.json')

loading annotations into memory...
Done (t=1.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [131]:
# 转换成proposal
train_proposals = cvt_ins_ann2proposal(train_ins)
test_proposals  = cvt_ins_ann2proposal( test_ins)

  8%|▊         | 2952/37633 [00:00<00:01, 29512.50it/s]

100%|██████████| 9546/9546 [00:00<00:00, 28599.53it/s]


In [ ]:
# train_proposals = load(train_total_prop_path)
# test_proposals  = load( test_total_prop_path)
# len(train_proposals), len(test_proposals)

### person_keypoints

In [132]:
# 提取仅含“人”的实例
train_person, train_imgs_wo_human = extract_person_from_proposal(train_proposals, train_img)
test_person ,  test_imgs_wo_human = extract_person_from_proposal( test_proposals,  test_img)
dump(train_person, train_person_path)
dump( test_person,  test_person_path)

100%|██████████| 37633/37633 [00:01<00:00, 23075.09it/s]


num person:  109336
num image wo person:  0


100%|██████████| 9546/9546 [00:00<00:00, 31995.87it/s]

num person:  30896
num image wo person:  0


In [41]:
# train_person = load(train_person_path)
# test_person  = load( test_person_path)
# len(train_person), len(test_person)

(109336, 30896)

In [42]:
print(train_person[100])

{'id': 196, 'bbox': [274, 45, 98, 170], 'area': 16660, 'category_id': 1, 'image_id': 44, 'score': 1.0}


In [139]:
# 获取姿态估计结果
train_pose = load(train_pose_prop_path)
test_pose  = load( test_pose_prop_path)
assert len(train_pose) == len(train_person), f'{len(train_pose)} != {len(train_person)}'
assert len( test_pose) == len( test_person), f'{len( test_pose)} != {len( test_person)}'

In [140]:
# 转换成keypoints_proposal
train_proposals = merge_pose_to_prop(train_proposals, train_pose, train_img)
test_proposals  = merge_pose_to_prop( test_proposals,  test_pose,  test_img)

100%|██████████| 109336/109336 [00:00<00:00, 503601.87it/s]


num image: 37633
num image wo pose prop: 0


100%|██████████| 30896/30896 [00:00<00:00, 495331.79it/s]


num image: 9546
num image wo pose prop: 0


In [103]:
dump(train_proposals, train_total_prop_path)
dump( test_proposals,  test_total_prop_path)